# Imports

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")  

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Acesso a los datos

In [ ]:
%cd "/content/gdrive/Shareddrives/Cortesanas de IA/Reto/Modelado_v2"
!ls

/content/gdrive/Shareddrives/Cortesanas de IA/Reto/Modelado_v2
antenas.csv		   NN_RetoV1.ipynb
datasets		   NN_RetoV2.ipynb
final.csv		   obtain_data_Chile.ipynb
Get_%_of_homeoffice.ipynb  spark-3.2.2-bin-hadoop3.2
Getting_model.ipynb	   spark-3.2.2-bin-hadoop3.2.tgz
HomeOffice.csv		   spark-3.2.2-bin-hadoop3.2.tgz.1
merge_data_Chile.ipynb	   SparkML.ipynb
new_train.csv


# PySpark

## Configuracion

In [ ]:
#Bibliotecas para poder trabajar con Spark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://downloads.apache.org/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz  
#Configuración de Spark con Python
!pip install -q findspark
!pip install pyspark

#Estableciendo variable de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.2-bin-hadoop3.2"

#Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Building dependency tree       
Reading state infor

'spark-3.2.2-bin-hadoop3.2'

Now, we can import SparkSession from pyspark.sql and create a SparkSession, which is the entry point to Spark.

You can give a name to the session using appName() and add some configurations with config() if you wish.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("SparkML")\
        .getOrCreate()

In [ ]:
spark

## Loading data into Spark

In [ ]:
df = spark.read.csv("final.csv", header=True, inferSchema=True)

## Exploring Data

In [ ]:
# Show column detail
df.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- %_of_homeoffice: double (nullable = true)
 |-- Cantidad de Conexiones de internet fijas: double (nullable = true)
 |-- Numero de empresas sin ventas: double (nullable = true)
 |-- Cantidad de trabajadores en empresas sin ventas: double (nullable = true)
 |-- Numero de empresas Micro 1: double (nullable = true)
 |-- Cantidad de trabajadores en empresas Micro 1: double (nullable = true)
 |-- Numero de empresas Pequeña 1: double (nullable = true)
 |-- Cantidad de trabajadores en empresas Pequeña 1: double (nullable = true)
 |-- Numero de empresas Mediana 1: double (nullable = true)
 |-- Cantidad de trabajadores en empresas Mediana 1: double (nullable = true)
 |-- Numero de empresas Grande 1: double (nullable = true)
 |-- Cantidad de trabajadores en empresas Grande 1: double (nullable = true)
 |-- Consumo de Electricidad en Servicios Comunitarios en M$: string (nullable = true)
 |-- Consumo de Electricidad en M$: string (nullable = true)


In [ ]:
# Display Rows
df.show(5)

+----------------+------------------+----------------------------------------+-----------------------------+-----------------------------------------------+--------------------------+--------------------------------------------+----------------------------+----------------------------------------------+----------------------------+----------------------------------------------+---------------------------+---------------------------------------------+-------------------------------------------------------+-----------------------------+------------------------------------------------------+---------------------+------------------------+-------------------------------------+----------------------------------------+
|      Unnamed: 0|   %_of_homeoffice|Cantidad de Conexiones de internet fijas|Numero de empresas sin ventas|Cantidad de trabajadores en empresas sin ventas|Numero de empresas Micro 1|Cantidad de trabajadores en empresas Micro 1|Numero de empresas Pequeña 1|Cantidad de trabajado

In [ ]:
# Describing the columns
df.describe().show()

+-------+----------+------------------+----------------------------------------+-----------------------------+-----------------------------------------------+--------------------------+--------------------------------------------+----------------------------+----------------------------------------------+----------------------------+----------------------------------------------+---------------------------+---------------------------------------------+-------------------------------------------------------+-----------------------------+------------------------------------------------------+---------------------+------------------------+-------------------------------------+----------------------------------------+
|summary|Unnamed: 0|   %_of_homeoffice|Cantidad de Conexiones de internet fijas|Numero de empresas sin ventas|Cantidad de trabajadores en empresas sin ventas|Numero de empresas Micro 1|Cantidad de trabajadores en empresas Micro 1|Numero de empresas Pequeña 1|Cantidad de traba

## Preprocessing Data

In [ ]:
from pyspark.sql.functions import col
df = df.withColumn("Consumo de Electricidad en Servicios Comunitarios en M$",col("Consumo de Electricidad en Servicios Comunitarios en M$").cast('double'))
df = df.withColumn("Consumo de Electricidad en M$",col("Consumo de Electricidad en M$").cast('double'))
df = df.withColumn("Consumo de Electricidad Dependencias Municipales en M$",col("Consumo de Electricidad Dependencias Municipales en M$").cast('double'))

In [ ]:
# Show column detail
df.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- %_of_homeoffice: double (nullable = true)
 |-- Cantidad de Conexiones de internet fijas: double (nullable = true)
 |-- Numero de empresas sin ventas: double (nullable = true)
 |-- Cantidad de trabajadores en empresas sin ventas: double (nullable = true)
 |-- Numero de empresas Micro 1: double (nullable = true)
 |-- Cantidad de trabajadores en empresas Micro 1: double (nullable = true)
 |-- Numero de empresas Pequeña 1: double (nullable = true)
 |-- Cantidad de trabajadores en empresas Pequeña 1: double (nullable = true)
 |-- Numero de empresas Mediana 1: double (nullable = true)
 |-- Cantidad de trabajadores en empresas Mediana 1: double (nullable = true)
 |-- Numero de empresas Grande 1: double (nullable = true)
 |-- Cantidad de trabajadores en empresas Grande 1: double (nullable = true)
 |-- Consumo de Electricidad en Servicios Comunitarios en M$: double (nullable = true)
 |-- Consumo de Electricidad en M$: double (nullable = true)


In [ ]:
df = df.na.fill(0)

In [ ]:
# Describing the columns
df.describe().show()

+-------+----------+------------------+----------------------------------------+-----------------------------+-----------------------------------------------+--------------------------+--------------------------------------------+----------------------------+----------------------------------------------+----------------------------+----------------------------------------------+---------------------------+---------------------------------------------+-------------------------------------------------------+-----------------------------+------------------------------------------------------+---------------------+------------------------+-------------------------------------+----------------------------------------+
|summary|Unnamed: 0|   %_of_homeoffice|Cantidad de Conexiones de internet fijas|Numero de empresas sin ventas|Cantidad de trabajadores en empresas sin ventas|Numero de empresas Micro 1|Cantidad de trabajadores en empresas Micro 1|Numero de empresas Pequeña 1|Cantidad de traba

## Models

In [ ]:
modelsResults = [['Model','RMSE','MSE','MAE','R2']]

In [ ]:
# Imports
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import VectorAssembler

In [ ]:
# Using VectorAssembler
assembler = VectorAssembler(inputCols = ['Cantidad de Conexiones de internet fijas', 'Numero de empresas sin ventas', 'Cantidad de trabajadores en empresas sin ventas', 'Numero de empresas Micro 1', 'Cantidad de trabajadores en empresas Micro 1', 'Numero de empresas Pequeña 1','Cantidad de trabajadores en empresas Pequeña 1','Numero de empresas Mediana 1','Cantidad de trabajadores en empresas Mediana 1','Numero de empresas Grande 1','Cantidad de trabajadores en empresas Grande 1','Consumo de Electricidad en Servicios Comunitarios en M$','Consumo de Electricidad en M$','Consumo de Electricidad Dependencias Municipales en M$','Empresas informaticas','Empresas no informaticas','Trabajadores en empresas informaticas','Trabajadores en empresas no informaticas'], outputCol='features')
output = assembler.transform(df)

In [ ]:
output.show()

+-----------------+------------------+----------------------------------------+-----------------------------+-----------------------------------------------+--------------------------+--------------------------------------------+----------------------------+----------------------------------------------+----------------------------+----------------------------------------------+---------------------------+---------------------------------------------+-------------------------------------------------------+-----------------------------+------------------------------------------------------+---------------------+------------------------+-------------------------------------+----------------------------------------+--------------------+
|       Unnamed: 0|   %_of_homeoffice|Cantidad de Conexiones de internet fijas|Numero de empresas sin ventas|Cantidad de trabajadores en empresas sin ventas|Numero de empresas Micro 1|Cantidad de trabajadores en empresas Micro 1|Numero de empresas Pequeña 

In [ ]:
# Selecting the input and output columns for modeling
finalised_data = output.select('%_of_homeoffice','features')
finalised_data.show()

+------------------+--------------------+
|   %_of_homeoffice|            features|
+------------------+--------------------+
| 61.26060706653665|[123359.0,13507.0...|
|58.401903053464885|[171687.0,15587.0...|
| 59.68975311339305|[5635.0,306.0,804...|
| 59.54130366327281|[52799.0,1410.0,1...|
| 60.14623850444746|[39371.0,2102.0,2...|
| 59.12806539509537|[41276.0,1288.0,1...|
|  61.9877800407332|[22192.0,981.0,19...|
|  59.7819850831899|[61302.0,1827.0,2...|
| 62.19272369714847|[28820.0,1509.0,2...|
|62.821141057052856|[29817.0,976.0,33...|
| 61.35646687697161|[27202.0,2248.0,2...|
| 62.48118269193654|[158938.0,4554.0,...|
| 61.83663283979371|[21439.0,1152.0,9...|
| 61.85176414870945|[51909.0,1788.0,3...|
|53.756994404476416|[873.0,229.0,1565...|
|   60.703081232493|[115763.0,3939.0,...|
|  58.6189683860233|[17607.0,585.0,11...|
| 61.22916666666666|[34864.0,3078.0,4...|
| 63.52806995311114|[13601.0,495.0,46...|
| 62.53512401265571|[153696.0,4301.0,...|
+------------------+--------------

###OLS

In [ ]:
import pandas as pd
import numpy as np
df1 = pd.read_csv('final.csv')
data = df1.replace(' -', 0)
data3 = data.drop(["Unnamed: 0"], axis=1)
data4 = np.asarray(data3).astype(np.float32)
df = pd.DataFrame(data4, columns = ['Y','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r'])
df.head()

,Y,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r
0,61.260609,123359.0,13507.0,15727.0,11934.0,2533.0,6200.0,17716.0,2102.0,44293.0,980.0,68487.0,1608421.0,375540.0,1232881.0,32101.0,33332.0,368675.0,651272.0
1,58.401901,171687.0,15587.0,207904.0,16220.0,59506.0,5769.0,26065.0,1449.0,52774.0,424.0,54314.0,3035099.0,420817.0,2614282.0,23551.0,47660.0,367621.0,711310.0
2,59.689754,5635.0,306.0,804.0,491.0,1478.0,121.0,386.0,15.0,593.0,3.0,125.0,227620.0,227620.0,0.0,221.0,1466.0,192.0,5683.0
3,59.541306,52799.0,1410.0,1737.0,2130.0,358.0,595.0,3050.0,118.0,6091.0,43.0,12435.0,1807185.0,68539.0,1738646.0,1640.0,6324.0,10333.0,61895.0
4,60.146240,39371.0,2102.0,2038.0,3161.0,503.0,1095.0,4811.0,353.0,8464.0,99.0,5706.0,602111.0,183891.0,417268.0,2891.0,10288.0,25591.0,52594.0


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
mod = smf.ols('Y ~ a + b + c + d + e + f + g + h + i + j + k + l + m + n + o + p + q + r', data = df).fit()

print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.138
Model:                            OLS   Adj. R-squared:                 -0.332
Method:                 Least Squares   F-statistic:                    0.2941
Date:                Thu, 01 Dec 2022   Prob (F-statistic):              0.996
Time:                        02:51:03   Log-Likelihood:                -136.04
No. Observations:                  52   AIC:                             310.1
Df Residuals:                      33   BIC:                             347.2
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     60.2216      1.348     44.660      0.0

### Linear regression

In [ ]:
#Imports
from pyspark.ml.regression import LinearRegression

In [ ]:
# Splitting the data
train, test = finalised_data.randomSplit([0.8, 0.2])

In [ ]:
# Create Model
lr = LinearRegression(labelCol="%_of_homeoffice", featuresCol="features")

In [ ]:
# create paramGrid
lrparamGrid = (ParamGridBuilder()
             .addGrid(lr.maxIter, [5, 10, 20, 50, 100])
             .build())

In [ ]:
# create evaluator
eval = RegressionEvaluator(labelCol="%_of_homeoffice", predictionCol="prediction", metricName="rmse")

In [ ]:
# Create 5-fold CrossValidator
lrcv = CrossValidator(estimator = lr,
                    estimatorParamMaps = lrparamGrid,
                    evaluator = eval,
                    numFolds = 5)

In [ ]:
# Fit the model
lrModel = lrcv.fit(train)

In [ ]:
# Predict
lr_predictions = lrModel.transform(test)

In [ ]:
lr_predictions.show()

+------------------+--------------------+------------------+
|   %_of_homeoffice|            features|        prediction|
+------------------+--------------------+------------------+
| 56.73789173789174|[34980.0,986.0,40...|61.193383682428795|
|59.055385660890245|[30440.0,1251.0,1...| 63.94597945097816|
| 59.34154175588865|[23731.0,886.0,66...| 60.32789355811679|
| 59.66001628664496|[15156.0,908.0,26...|58.821832941363766|
| 59.92915190664722|[2657.0,378.0,294...| 60.14958698598768|
| 60.14623850444746|[39371.0,2102.0,2...| 61.11575469146424|
|  60.3170755357316|[19190.0,1529.0,1...|  64.2907593896874|
|   60.703081232493|[115763.0,3939.0,...| 67.24554935616229|
| 61.35646687697161|[27202.0,2248.0,2...| 62.63956434633898|
|  61.9877800407332|[22192.0,981.0,19...| 59.70139371717927|
| 62.00632684342693|[69448.0,2798.0,4...|  65.9411354494238|
|62.821141057052856|[29817.0,976.0,33...| 65.20875120005043|
| 62.85784494409633|[23271.0,804.0,11...| 59.66799694766178|
+------------------+----

In [ ]:
# EVALUATE


# RMSE
rmse = eval.evaluate(lr_predictions)
print("RMSE: %.3f" % rmse)

#  MSE
mse = eval.evaluate(lr_predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# MAE
mae = eval.evaluate(lr_predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# R2
r2 = eval.evaluate(lr_predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 3.320
MSE: 11.020
MAE: 2.766
r2: -3.097


In [ ]:
# Save model results
result_lr= ['Linear regression', rmse, mse, mae, r2]
modelsResults.append(result_lr)

### Gradient-Boosted Trees (GBTs)

#### Modelo inicial

In [ ]:
# Imports
from pyspark.ml.regression import GBTRegressor

In [ ]:
# Splitting the data
train, test = finalised_data.randomSplit([0.8, 0.2])

In [ ]:
# Create Model
gb = GBTRegressor(labelCol="%_of_homeoffice", featuresCol="features")

In [ ]:
# create paramGrid
gbparamGrid = (ParamGridBuilder()
             .addGrid(gb.maxDepth, [6, 12, 18, 24, 30])
             .addGrid(gb.maxBins, [18, 36, 72, 144, 288])
             .addGrid(gb.maxIter, [5, 10, 20, 50, 100])
             .build())

In [ ]:
# create evaluator
eval = RegressionEvaluator(labelCol="%_of_homeoffice", predictionCol="prediction", metricName="rmse")

In [ ]:
# Create 5-fold CrossValidator
gbcv = CrossValidator(estimator = gb,
                    estimatorParamMaps = gbparamGrid,
                    evaluator = eval,
                    numFolds = 5)

In [ ]:
# Fit the model
gbModel = gbcv.fit(train)

In [ ]:
# Predict
gb_predictions = gbModel.transform(test)

In [ ]:
gb_predictions.show()

+-----------------+--------------------+------------------+
|  %_of_homeoffice|            features|        prediction|
+-----------------+--------------------+------------------+
|59.12806539509537|[41276.0,1288.0,1...|58.891540250559686|
|59.50530035335689|[36837.0,1437.0,4...| 59.24284982347472|
|60.14623850444746|[39371.0,2102.0,2...| 61.58016855514044|
| 60.3170755357316|[19190.0,1529.0,1...| 62.28296658923037|
|60.40983606557377|[24179.0,806.0,34...| 58.64190699153327|
|  60.703081232493|[115763.0,3939.0,...| 62.21875150322616|
|60.91410559495666|[13845.0,557.0,59...| 63.21018020939858|
|61.22916666666666|[34864.0,3078.0,4...| 61.58016855514044|
| 62.7673576834485|[26354.0,736.0,15...| 59.75352656958351|
|62.85784494409633|[23271.0,804.0,11...| 58.35914599345427|
| 63.0410198416778|[20606.0,1290.0,2...| 60.15578686108455|
|73.49726775956285|[411.0,155.0,547....|53.740281708471606|
+-----------------+--------------------+------------------+



In [ ]:
# Evaluate


# RMSE
rmse = eval.evaluate(gb_predictions)
print("RMSE: %.3f" % rmse)

#  MSE
mse = eval.evaluate(gb_predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# MAE
mae = eval.evaluate(gb_predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# R2
r2 = eval.evaluate(gb_predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 6.089
MSE: 37.071
MAE: 3.332
r2: -1.758


In [ ]:
# Save model results
result_gb= ['GBT 1.0', rmse, mse, mae, r2]
modelsResults.append(result_gb)